##### １：webdriverとsleepをインポート

In [1]:
from selenium import webdriver
from time import sleep

##### ２：ユーザーネーム、パスワードを入力

In [2]:
USERNAME = "username"
PASSWORD = "password"

##### ３：インスタグラムのURLを読み込む

In [3]:
driver = webdriver.Chrome(r"C:\Users\user\chromedriver-win64\chromedriver-win64\chromedriver.exe")
target_url = "https://www.instagram.com/"
driver.get(target_url)
sleep(3)

##### ４：エラーの初期値をFalseに

In [4]:
error_flg = False

##### ５：自動でログインする処理

In [5]:
try:
    username_input = driver.find_element_by_xpath("//input[@aria-label='電話番号、ユーザーネーム、メールアドレス']")
    username_input.send_keys(USERNAME)
    sleep(2) 

    password_input = driver.find_element_by_xpath("//input[@aria-label='パスワード']")
    password_input.send_keys(PASSWORD)
    sleep(2)

    login_button = driver.find_element_by_xpath("//button[@type='submit']")
    login_button.submit()
    sleep(2)
    
except Exception:
    error_flg = True
    print("エラーが発生しました")

##### ６：「後で」ボタンを押す処理（２回）

In [6]:
if error_flg is False:
    try:
        sleep(2)
        notnow_button = driver.find_element_by_xpath("//div[text()='後で']")
        notnow_button.click()
        sleep(2)
        notnow_button = driver.find_element_by_xpath("//button[text()='後で']")
        notnow_button.click()
        sleep(2)
    except Exception:
        pass

##### ６：対象のユーザーを検索

In [7]:
target_username = "paulnicklen"
if error_flg is False:
    try:
         target_profile_url = target_url + target_username
         driver.get(target_profile_url)
         sleep(2)
    except Exception:
         print("検索時にエラーが発生しました")
         error_flg = True

#####  ７：スクロールして画像URLを取得 & 出力

In [8]:
from bs4 import BeautifulSoup
if error_flg is False:
    try:
        post_count = 24
        if post_count > 12:
            scroll_count = int(post_count/12) + 1
            try:
                all_images = []
                for i in range(scroll_count):
                    soup = BeautifulSoup(driver.page_source,"html.parser")
                    for image_tag in soup.select("div._aagv"):
                        for image in image_tag.find_all("img"):
                            all_images.append(image)
                    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
                    sleep(2)
                    if i > 2:
                        break
                    all_images = list(dict.fromkeys(all_images))
                for index, image in enumerate(all_images):
                    print("画像番号" + str(index))
                    print("image['src']:" + image['src'],end="\n\n")
            except Exception:
                print("画面スクロール中にエラーが発生しました")
                error_flg = True
    except Exception:
        print("投稿数が取得できませんでした")
        error_flg = True

画像番号0
image['src']:https://scontent.cdninstagram.com/v/t51.2885-15/426226065_215589208301828_671695173812292680_n.jpg?stp=dst-jpg_e15&efg=eyJ2ZW5jb2RlX3RhZyI6ImltYWdlX3VybGdlbi4xMDgweDE5MjAuc2RyIn0&_nc_ht=scontent.cdninstagram.com&_nc_cat=1&_nc_ohc=2CvJrmR40WsAX_4424O&edm=APs17CUBAAAA&ccb=7-5&ig_cache_key=MzI5ODMzMDE3MDU5MjY0OTMzNg%3D%3D.2-ccb7-5&oh=00_AfDZCm0idomEh_iYE33rQYM07P0x2wY213zEG3p5DDMiNA&oe=65C92D2B&_nc_sid=10d13b

画像番号1
image['src']:https://scontent.cdninstagram.com/v/t51.2885-15/425456276_352584947655224_2453043674621109670_n.jpg?stp=dst-jpg_e15&efg=eyJ2ZW5jb2RlX3RhZyI6ImltYWdlX3VybGdlbi43MjB4MTI4MC5zZHIifQ&_nc_ht=scontent.cdninstagram.com&_nc_cat=108&_nc_ohc=kXtpqqpGqzAAX-PR2qz&edm=APs17CUBAAAA&ccb=7-5&ig_cache_key=MzI5ODMyNzg2MjcwMjc3ODMyMQ%3D%3D.2-ccb7-5&oh=00_AfBcbCHbUeTkzTJMaleaI-5X1O1T9Ef1rwEcd3BjBeWkOw&oe=65C99FDF&_nc_sid=10d13b

画像番号2
image['src']:https://scontent.cdninstagram.com/v/t51.2885-15/425550615_764782985121270_7559801002357420663_n.jpg?stp=dst-jpg_e35&efg

##### ８：ローカルに保存する処理

In [9]:
import requests
import re
import os
import shutil

path = r"C:\Users\user\OneDrive\picture\Photo"

if error_flg is False:
    try:
        for index, image in enumerate(all_images):
            filename = "image_" + str(index) + ".jpg"
            image_path = os.path.join(path, filename)
            image_link = image["src"]
    
            url_ptn = re.compile(r"(http|https)://")
            res = url_ptn.match(image_link)
            if res:
                response = requests.get(image_link,stream=True)
                with open(image_path, "wb") as file:
                    shutil.copyfileobj(response.raw, file)
    except Exception as e:
        print(e)
        print(str(index) + "番目の画像のダウンロードに失敗しました")
        print("画像へのリンク：" + image_link)